<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 02 Adding New Variables

### Riccardo Spolaor (864877)

</center>

### Creazione Variabili aggiuntive

In [223]:
import numpy as np

In [256]:
%store -r
%store -z

In [257]:
dataframe.head()

,csvID,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,...,WPts,WPtsAdded,LPts,LPtsAdded,MaxW,MaxL,AvgW,AvgL,MinW,MinL
0,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,...,1193.314121,1,329.750000,1,1.28,3.80,1.2000,3.550,1.12,3.30
1,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,2nd Round,3.0,...,1383.760192,1,745.688172,1,1.35,3.20,1.2825,2.925,1.20,2.60
2,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,2nd Round,3.0,...,1193.314121,1,543.656627,1,1.13,5.50,1.1300,5.500,1.13,5.50
3,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,2nd Round,3.0,...,733.040936,1,659.596567,1,2.70,1.45,2.4075,1.430,2.00,1.41
4,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,Quarterfinals,3.0,...,1383.760192,1,1193.314121,1,2.30,1.53,2.1000,1.515,1.90,1.50


In [258]:
grouped = dataframe.groupby(['csvID', 'ATP', 'Tournament'], sort = False)
uniquelists = []
for c in grouped.groups.keys():
    uniquelists += [tuple(t) for t in [grouped.get_group(c)['Round'].unique()]]
    
print(set(uniquelists))
# I campionati sono ordinati in base al round

{('1st Round', '2nd Round', 'Quarterfinals', 'Semifinals', 'The Final'), ('1st Round', '2nd Round', '3rd Round', '4th Round'), ('1st Round', '2nd Round', 'Semifinals', 'The Final'), ('1st Round', '2nd Round', 'Semifinals'), ('1st Round', '2nd Round', 'Quarterfinals'), ('1st Round', 'Quarterfinals'), ('1st Round', '2nd Round', 'Quarterfinals', 'The Final'), ('1st Round', '2nd Round', '3rd Round', 'Quarterfinals', 'Semifinals'), ('1st Round', '2nd Round', '3rd Round', 'Quarterfinals', 'Semifinals', 'The Final'), ('Round Robin', 'Quarterfinals', 'Semifinals', 'The Final'), ('2nd Round', 'Quarterfinals', 'The Final'), ('1st Round', '2nd Round', '3rd Round', 'Quarterfinals'), ('1st Round', '2nd Round', '3rd Round', '4th Round', 'Quarterfinals', 'Semifinals'), ('Round Robin', 'Semifinals', 'The Final'), ('1st Round', '2nd Round'), ('1st Round', '2nd Round', '3rd Round', '4th Round', 'Quarterfinals', 'Semifinals', 'The Final'), ('Round Robin', 'Semifinals'), ('1st Round', '2nd Round', '3rd Ro

In [259]:
# Itero una volta aggiungendo la feature delle partite giocate da perdente e vincitore finora
def setPlayedMatches ():
    wPlayedIndex = dataframe.columns.get_loc('WPlayed')
    lPlayedIndex = dataframe.columns.get_loc('LPlayed')
    playersDictionary = {p:0 for p in dataframe['Winner'].append(dataframe['Loser']).unique()}
    for i,r in dataframe.iterrows():
        dataframe.iloc[i,wPlayedIndex] = playersDictionary[r['Winner']]
        dataframe.iloc[i,lPlayedIndex] = playersDictionary[r['Loser']]
        playersDictionary[r['Loser']] += 1
        playersDictionary[r['Winner']] += 1

In [260]:
dataframe.insert(dataframe.columns.get_loc('Loser') + 1, column = 'WPlayed', value = np.nan)
dataframe.insert(dataframe.columns.get_loc('WPlayed') + 1, column = 'LPlayed', value = np.nan)

In [261]:
setPlayedMatches()

In [262]:
dataframe[['Winner', 'Loser', 'WPlayed', 'LPlayed']].head()

,Winner,Loser,WPlayed,LPlayed
0,Hewitt L.,Arthurs W.,0.0,0.0
1,Haas T.,Malisse X.,0.0,0.0
2,Hewitt L.,Phau B.,1.0,0.0
3,Massu N.,Clement A.,0.0,0.0
4,Haas T.,Hewitt L.,1.0,2.0


In [263]:
def getPlayerWinLossRatio ():
    
    wWinRatioIndex = dataframe.columns.get_loc('WWinRatio')
    lWinRatioIndex = dataframe.columns.get_loc('LWinRatio')
    wLossRatioIndex = dataframe.columns.get_loc('WLossRatio')
    lLossRatioIndex = dataframe.columns.get_loc('LLossRatio')
    
    winnersDictionary = {p:0 for p in dataframe['Winner'].append(dataframe['Loser']).unique()}
    losersDictionary = {p:0 for p in dataframe['Winner'].append(dataframe['Loser']).unique()}
    
    for i,r in dataframe.iterrows():
        if r['WPlayed'] == 0:
            wPlayed = 1
        else:
            wPlayed = r['WPlayed']
            
        if r['LPlayed'] == 0:
            lPlayed = 1
        else:
            lPlayed = r['LPlayed']
            
        
        dataframe.iloc[i,wWinRatioIndex] = winnersDictionary[r['Winner']]/wPlayed
        dataframe.iloc[i,lWinRatioIndex] = winnersDictionary[r['Loser']]/lPlayed
        dataframe.iloc[i,wLossRatioIndex] = losersDictionary[r['Winner']]/wPlayed
        dataframe.iloc[i,lLossRatioIndex] = losersDictionary[r['Loser']]/lPlayed
        
        winnersDictionary[r['Winner']] += 1
        losersDictionary[r['Loser']] += 1

In [264]:
dataframe.insert(dataframe.columns.get_loc('LPlayed') + 1, column = 'WWinRatio', value = np.nan)
dataframe.insert(dataframe.columns.get_loc('WWinRatio') + 1, column = 'LWinRatio', value = np.nan)
dataframe.insert(dataframe.columns.get_loc('LWinRatio') + 1, column = 'WLossRatio', value = np.nan)
dataframe.insert(dataframe.columns.get_loc('WLossRatio') + 1, column = 'LLossRatio', value = np.nan)

In [266]:
getPlayerWinLossRatio()

In [269]:
dataframe[['Winner', 'Loser', 'WPlayed', 'LPlayed', 'WWinRatio', 'LWinRatio', 'WLossRatio', 'LLossRatio']].head()

,Winner,Loser,WPlayed,LPlayed,WWinRatio,LWinRatio,WLossRatio,LLossRatio
0,Hewitt L.,Arthurs W.,0.0,0.0,0.0,0.0,0.0,0.0
1,Haas T.,Malisse X.,0.0,0.0,0.0,0.0,0.0,0.0
2,Hewitt L.,Phau B.,1.0,0.0,1.0,0.0,0.0,0.0
3,Massu N.,Clement A.,0.0,0.0,0.0,0.0,0.0,0.0
4,Haas T.,Hewitt L.,1.0,2.0,1.0,1.0,0.0,0.0


In [270]:
%store dataframe
%store

Stored 'dataframe' (DataFrame)
Stored variables and their in-db values:
dataframe             ->        csvID   ATP  Location            Tournament
